# Generating Subsets of Wikidata

>Warning: 
**This notebook is under construction and it doesn't work**

## Purpose

>This notebook is used to create smaller subgraphs from a larger input Wikidata graph. Notebook users can provide a list of Wikidata classes (**QNodes**) to remove and preserve to create desired subsets of Wikidata. 


### Batch Invocation
Example batch command. The second argument is a notebook where the output will be stored. You can load it to see progress.

UPDATE EXAMPLE INVOCATION


```
papermill Wikidata\ Useful\ Files.ipynb useful-files.out.ipynb \
-p wiki_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/all.tsv.gz \
-p label_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/part.label.en.tsv.gz \
-p item_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/part.wikibase-item.tsv.gz \
-p property_item_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/part.property.wikibase-item.tsv.gz \
-p qual_file /Volumes/GoogleDrive/Shared\ drives/KGTK-public-graphs/wikidata-20200803-v3/qual.tsv.gz \
-p output_path <local folder> \
-p output_folder useful_files_v4 \
-p temp_folder temp.useful_files_v4 \
-p delete_database no \
-p compute_pagerank no \
-p languages es,ru,zh-cn 
```

In [11]:
# Parameters

# Folder on local machine where to create the output and temporary folders
# output_path = "/Users/pedroszekely/Downloads/kypher"
output_path = "/Users/markmann/Downloads/subset"

# The names of the output and temporary folders
output_folder = "output"
temp_folder = "temp.output"

# Classes to remove
#Q34508 - video tape recording
# remove_classes = "Q13442814, Q523, Q16521, Q318, Q7318358, Q7187, Q11173, Q8054, Q5, Q13100073, Q8502, Q3305213, Q4022, Q79007, Q1931185, Q30612, Q101352, Q54050, Q13433827, Q2668072, Q23397, Q3863, Q11424, Q482994, Q47150325, Q16970, Q18593264, Q355304, Q9842, Q7725634, Q27020041, Q56436498, Q2154519, Q61443690, Q49008, Q3331189, Q47521, Q5084, Q19389637, Q21014462, Q4164871, Q11060274, Q5633421, Q39816, Q5185279, Q55488, Q134556, Q22698, Q985488, Q1260524, Q204107, Q2225692, Q215380, Q71963409, Q452237, Q93184, Q12323"
remove_classes = "Q34508"

# The location of input files
# wiki_root_folder = "/Volumes/GoogleDrive/Shared\ drives/KGTK/datasets/wikidata-20200803-v4/"
# wiki_root_folder = "/Volumes/GoogleDrive/Shared\ drives/KGTK/datasets/wikidata-20200803-v4/"
# wiki_root_folder = "/Users/pedroszekely/Downloads/kypher/wikidataos-v4/"
wiki_root_folder = "/Users/markmann/Google\ Drive/Shared\ drives/KGTK/datasets/wikidataos-v4-mm-2/"

claims_file = "claims.tsv.gz"
label_file = "labels.en.tsv.gz"
alias_file = "aliases.en.tsv.gz"
description_file = "descriptions.en.tsv.gz"
item_file = "claims.wikibase-item.tsv.gz"
qual_file = "qualifiers.tsv.gz"
property_datatypes_file = "metadata.property.datatypes.tsv.gz"
metadata_file = "metadata.types.tsv.gz" #Add
isa_file = "derived.isa.tsv.gz"
p279star_file = "derived.P279star.tsv.gz"
out_degree_file = "metadata.out_degree.tsv.gz"

# Useful files Jupyter notebook
useful_files_notebook = "Wikidata Useful Files.ipynb"
notebooks_folder = "/Users/markmann/Desktop/CKG/kgtk_subset/kgtk/examples/"

# Location of the cache database for kypher
# cache_path = "/Users/pedroszekely/Downloads/kypher/wikidataos-v4"
cache_path = f'{output_path}/{output_folder}'

#Additional parameters
delete_database = "no"
compute_pagerank = "no"
languages = ""

### Needs fixing
# Whether to delete the cache database
if delete_database and delete_database.lower().strip() == 'yes':
    delete_database = True
else:
    delete_database = False

### Needs fixing
if compute_pagerank and compute_pagerank.lower().strip() == 'yes':
    compute_pagerank = True
else:
    compute_pagerank = False

if languages:
    languages = languages.split(',')

In [12]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd

import altair as alt

import papermill as pm

import gzip

## Set up variables for files

In [13]:
#Environment variables
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)

#Python variables
if cache_path:
    store = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    store = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)

out = "{}/{}".format(output_path, output_folder)
temp = "{}/{}".format(output_path, temp_folder)

claims = wiki_root_folder + claims_file
labels = wiki_root_folder + label_file
aliases = wiki_root_folder + alias_file
descriptions = wiki_root_folder + description_file
items = wiki_root_folder + item_file
quals = wiki_root_folder + qual_file
datatypes = wiki_root_folder + property_datatypes_file
metadata = wiki_root_folder + metadata_file #Add
isa = wiki_root_folder + isa_file
p279star = wiki_root_folder + p279star_file
out_degree = wiki_root_folder + out_degree_file

# shortcuts to commands
kgtk = "time kgtk --debug"
kypher = "kgtk query --debug --graph-cache " + store

In [ ]:
#Check files
#Example
#Q34508 - video tape recording

# !gzcat {p279star} | head
# !gzcat {descriptions} | head

# !zgrep 'Q34508' {datatypes}
# !zgrep '\Q.*' {datatypes}
# !zgrep 'Q34508' {claims}

Go to the output directory and create the subfolders for the output files and the temporary files

In [4]:
!cd $output_path
!mkdir {out}
!mkdir {temp}

mkdir: /Users/markmann/Downloads/subset/output: File exists
mkdir: /Users/markmann/Downloads/subset/temp.output: File exists


Clean up the output and temp folders before we start

In [ ]:
# !rm {out}/*.tsv {out}/*.tsv.gz
# !rm {temp}/*.tsv {temp}/*.tsv.gz

if delete_database:
    !rm {out}/*.tsv {out}/*.tsv.gz
    !rm {temp}/*.tsv {temp}/*.tsv.gz

In [ ]:
#COPY REFERENCE FILES
# !cp {datatypes} {out}
# !cp {metadata} {out}

### Preview the input files

It is always a good practice to peek a the files to make sure the column headings are what we expect

In [ ]:
!{kypher} -i {claims} \
--match '()-[]->()' \
--limit 10

## Creating a list of all the items  to remove

In [5]:
remove_list = set()

**Add classes to remove <u>directly</u> here:** <br>
- **Example:** Let's remove the class (videotape recording, 'Q34508')
- **NOTE:** This will only remove items that have a P31/P279 relation with the class

In [6]:
remove_classes_list = set()
classes_to_remove = ['Q34508'] #Parameter: Add classes manually here
[remove_classes_list.add(cl) for cl in classes_to_remove]

[None]

**Compute classes to remove, based on <u>instances</u> here:**
- **Example:** Let's remove classes that are part of instance (Fireball, 'Q5451712'), (Bush, 'Q1017471'), and (Italin Grape Alle, 'Q67772833')
- **NOTE:** The expected class to remove is (whisky, 'Q281'), (beer, 'Q44'), (beer brand, Q15075508), and (ale, 'Q208385')

In [8]:
#For each instance, find all classes associated with that instance
instances_to_remove = ['Q5451712', 'Q1017471', 'Q67772833'] #Parameter: Add instances manually here
instances = ', '.join([f'"{instance}"' for instance in instances_to_remove])

#Query for instancess
# !{kypher} -i {claims} \
!{kypher} -i {claims} -o {temp}/classes.remove.p31.tsv.gz \
--match '(instance)-[:P31]->(c)' \
--where 'instance in [{instances}]' \

#Query for subclasses
# !{kypher} -i {claims} \
!{kypher} -i {claims} -o {temp}/classes.remove.p279.tsv.gz \
--match '(instance)-[:P279]->(c)' \
--where 'instance in [{instances}]' \

#Concatenate into a single file
!kgtk cat -i {temp}/classes.remove.p31.tsv.gz -i {temp}/classes.remove.p279.tsv.gz \
-o {temp}/classes.remove.tsv.gz

# Deduplicate the returned classes - in place
!kgtk unique -i {temp}/classes.remove.tsv.gz \
-o {temp}/classes.remove.tsv.gz

# Extract classes and add to a list
fd = gzip.open(f'{temp}/classes.remove.tsv.gz', 'rt')
lines=fd.readlines()
[remove_classes_list.add(line.split('\t')[0]) for line in lines[1:]]
print(remove_classes_list)

[2021-02-09 13:56:50 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
     AND (graph_1_c1."node1" IN (?, ?, ?))
  PARAS: ['P31', 'Q5451712', 'Q1017471', 'Q67772833']
---------------------------------------------
[2021-02-09 13:56:51 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     WHERE graph_1_c1."label"=?
     AND (graph_1_c1."node1" IN (?, ?, ?))
  PARAS: ['P279', 'Q5451712', 'Q1017471', 'Q67772833']
---------------------------------------------
{'Q44', 'Q208385', 'Q15075508', 'Q281', 'Q34508'}


### Compute the items to be removed via classes

First look at the classes we will remove

In [9]:
classes = '"' + '", "'.join(list(remove_classes_list)) + '"'
print("classes to remove: ", classes)
# cmd = "wd u {}".format(" ".join(remove_classes.split(",")))
# !{cmd}

classes to remove:  "Q44", "Q208385", "Q15075508", "Q281", "Q34508"


Compose the kypher command to remove the classes

Run the command, the items to remove will be in file `{temp}/items.remove.byclass.tsv.gz`

In [10]:
!{kypher}  -i {isa} -i {p279star} -o {temp}/items.remove.byclass.tsv.gz \
--match 'isa: (n1)-[:isa]->(c), P279star: (c)-[]->(class)' \
--where 'class in [{classes}]' \
--return 'distinct n1, "p31_p279star" as label, class as node2'

[2021-02-09 13:57:11 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_2_c1."node1", ? "_aLias.label", graph_3_c2."node2" "_aLias.node2"
     FROM graph_2 AS graph_2_c1, graph_3 AS graph_3_c2
     WHERE graph_2_c1."label"=?
     AND graph_2_c1."node2"=graph_3_c2."node1"
     AND (graph_3_c2."node2" IN (?, ?, ?, ?, ?))
  PARAS: ['p31_p279star', 'isa', 'Q44', 'Q208385', 'Q15075508', 'Q281', 'Q34508']
---------------------------------------------


### Compute the items to be removed via out degree

Specify the # of node out-degrees (`degrees`), and remove those less than a certain degree

- **EXAMPLE:** Remove all items that have out-degree less than 5

In [ ]:
#Check out the file itself
# !zcat < {out_degree} | head -n 30 

degrees = 12 #Parameter

#TODO: Update with a larger file
# !{kypher} -i {temp}/metadata.out_degree.test.tsv.gz \ #Test
!{kypher} -i {out_degree} \
-o {temp}/items.remove.bydegree.tsv.gz \
--match '(n1)-[:out_degree]->(n2)' \
--where 'cast(n2, integer) <= {degrees}'

[2021-02-09 14:11:44 sqlstore]: IMPORT graph directly into table graph_8 from /Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4-mm-2/metadata.out_degree.tsv.gz ...


In [ ]:
#TODO: Check the byclass and bydgree files to ensure they have same header format
!zcat < {temp}/items.remove.byclass.tsv.gz | head 
!zcat < {temp}/items.remove.bydegree.tsv.gz | head 

#TODO: Cat the items.remove.byclass and items.remove.bydegree
# !kgtk cat -i {temp}/items.remove.byclass.tsv.gz {temp}/items.remove.bydegree.tsv.gz \
# -o {temp}/items.remove.tsv.gz

#TODO: Deduplicate the resulting concatenated file

Preview the `items.remove` file. Count number of class instances to remove

In [ ]:
# !zcat < {temp}/items.remove.tsv.gz | head | col

#Count of remove-list class Q34508 in claims file
# !zgrep 'Q34508' {claims} -c #466
# !zgrep '/tQ5/t' {claims} -c

#Count the remove-list class (Q34508) in items.remove.tsv
!zcat < {temp}/items.remove.tsv.gz
# !zgrep 'Q34508' {temp}/items.remove.tsv.gz -c #466

In [ ]:
!zcat < {temp}/items.remove.tsv.gz | wc

In [ ]:
# !zcat < {temp}/items.remove.tsv.gz | grep 'Q34508'

In [ ]:
# !zcat < {temp}/items.remove.tsv.gz | grep 'Q15874936\t'

Collect all the classes of items we will remove, just as a sanity check

In [ ]:
!{kypher} -i {temp}/items.remove.tsv.gz \
--match '()-[]->(n2)' \
--return 'distinct n2' \
--limit 10

## [TODO] Create a list of all items to protect

## Create the reduced edges file

### Remove the items from the all.tsv and the label, alias and description files
We will be left with `reduced` files where the edges do not have the unwanted items. We have to remove them from the node1 and node2 positions, so we need to run the ifnotexists commands twice.

Before we start preview the files to see the column headings and check whether they look sorted.

In [ ]:
!$kgtk sort2 -i {temp}/items.remove.tsv.gz -o {temp}/items.remove.sorted.tsv.gz

In [ ]:
!zcat < {temp}/items.remove.sorted.tsv.gz | head | col
# !zgrep 'Q34508' {temp}/items.remove.sorted.tsv.gz -c #466

In [ ]:
!zcat < {claims} | head -5 | col

Remove from the full set of edges those edges that have a `node1` present in `items.remove.sorted.tsv`

In [ ]:
!$kgtk ifnotexists -i {claims} -o {temp}/item.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted 

From the remaining edges, remove those that have a `node2` present in `items.remove.sorted.tsv`

In [ ]:
!$kgtk sort2 -i {temp}/item.edges.reduced.tsv.gz -o {temp}/item.edges.reduced.sorted.tsv.gz \
--columns node2 label node1 id

In [ ]:
!$kgtk ifnotexists -i {temp}/item.edges.reduced.sorted.tsv.gz -o {temp}/item.edges.reduced.2.tsv.gz \
--filter-on {temp}/items.remove.sorted.tsv.gz \
--input-keys node2 \
--filter-keys node1 \
--presorted 

Create a file with the labels

In [ ]:
!$kgtk ifnotexists -i {labels} -o {temp}/label.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted

In [ ]:
languages = ['en']

In [ ]:
#GOAL: Must sort the labels file

#NOT WORKING 
# for lang in languages:
#     cmd = f"kgtk sort2 -i {temp}/label.{lang}.edges.reduced.tsv.gz -o {out}/labels.{lang}.tsv.gz" 
#     !$cmd

#WORKING
!$kgtk sort2 -i {temp}/label.edges.reduced.tsv.gz -o {out}/labels.tsv.gz

Create a file with the aliases

In [ ]:
!$kgtk ifnotexists -i {aliases} -o {temp}/alias.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted

In [ ]:
#NOT WORKING: Languages issue
# for lang in languages:
#     cmd = f"kgtk --debug ifnotexists -i {wiki_root_folder}aliases.{lang}.tsv.gz \
#     -o {temp}/alias.{lang}.edges.reduced.tsv.gz \
#     --filter-on {temp}/items.remove.sorted.tsv.gz \
#     --input-keys node1 \
#     --filter-keys node1 \
#     --presorted"
#     !$cmd

In [ ]:
# for lang in languages:
#     cmd = f"kgtk sort2 -i {temp}/alias.{lang}.edges.reduced.tsv.gz -o {out}/aliases.{lang}.tsv.gz" 
#     !$cmd

Create a file with the descriptions

In [ ]:
!$kgtk ifnotexists -i {descriptions} -o {temp}/description.edges.reduced.tsv.gz \
--filter-on {temp}/items.remove.sorted.tsv.gz \
--input-keys node1 \
--filter-keys node1 \
--presorted

In [ ]:
# for lang in languages:
#     cmd = f"kgtk --debug ifnotexists -i {wiki_root_folder}descriptions.{lang}.tsv.gz \
#     -o {temp}/description.{lang}.edges.reduced.tsv.gz \
#     --filter-on {temp}/items.remove.sorted.tsv.gz \
#     --input-keys node1 \
#     --filter-keys node1 \
#     --presorted"
#     !$cmd

In [ ]:
# for lang in languages:
#     cmd = f"kgtk sort2 -i {temp}/description.{lang}.edges.reduced.tsv.gz -o {out}/descriptions.{lang}.tsv.gz" 
#     !$cmd

### Produce the output files for claims, labels, aliases and descriptions

In [ ]:
!$kgtk sort2 -i {temp}/item.edges.reduced.2.tsv.gz -o {out}/claims.tsv.gz 

In [ ]:
!$kgtk sort2 -i {temp}/label.edges.reduced.tsv.gz -o {out}/labels.en.tsv.gz 

In [ ]:
!$kgtk sort2 -i {temp}/alias.edges.reduced.tsv.gz -o {out}/aliases.en.tsv.gz 

In [ ]:
!$kgtk sort2 -i {temp}/description.edges.reduced.tsv.gz -o {out}/descriptions.en.tsv.gz 

Sanity checks to see if it looks reasonable

## Tests: Check each output file if class was removed / protected

In [ ]:
#test_1/remove-list/Q34508 removed? 
# !zcat < {out}/claims.tsv.gz | head | col
!zgrep 'Q34508' {temp}/item.edges.reduced.2.tsv.gz -c
# !zgrep 'Q34508' {out}/claims.tsv.gz -c #

## Create the reduced qualifiers file
We do this by finding all the ids of the reduced edges file, and then selecting out from `qual.tsv`

We need to join by id, so we need to sort both files by id, node1, label, node2:

- `{quals}` 
- `{out}/claims.tsv.gz` 

In [ ]:
!zcat < {quals} | head | column -t -s $'\t' 

Run `ifexists` to select out the quals for the edges in `{out}/wikidataos.qual.tsv.gz`. Note that we use `node1` in the qualifier file, matching to `id` in the `wikidataos.all.tsv` file.

In [ ]:
!$kgtk ifexists -i {quals} -o {out}/qualifiers.tsv.gz \
--filter-on {out}/claims.tsv.gz \
--input-keys node1 \
--filter-keys id \
--presorted

Look at the final output for qualifiers

In [ ]:
!zcat < {out}/qualifiers.tsv.gz | head | col

In [ ]:
# kgtk_path = "/Users/pedroszekely/Documents/GitHub/kgtk"
kgtk_path = "/Users/markmann/Desktop/CKG/kgtk_subset/kgtk"
os.environ["EXAMPLES_DIR"] = kgtk_path + "/examples"
os.environ["USECASE_DIR"] = kgtk_path + "/use-cases"
os.environ["TEMP"] = temp
os.environ["OUT"] = out
os.environ["DATATYPES"] = datatypes
os.environ["METADATA"] = metadata

In [ ]:
!ls "$TEMP"

In [ ]:
!ls "$OUT"

In [ ]:
!kgtk cat \
-i "$OUT"/aliases.en.tsv.gz \
-i "$OUT"/descriptions.en.tsv.gz \
-i "$OUT"/qualifiers.tsv.gz \
-i "$OUT"/claims.tsv.gz \
-i "$OUT"/labels.en.tsv.gz \
-i "$OUT"/metadata.property.datatypes.tsv.gz \
-i "$OUT"/metadata.types.tsv.gz \
-o "$OUT"/all.tsv.gz

In [ ]:
!ls {os.environ["EXAMPLES_DIR"] + "/partition-wikidata.ipynb"}

In [ ]:
pm.execute_notebook(
    os.environ["EXAMPLES_DIR"] + "/partition-wikidata.ipynb",
    os.environ["TEMP"] + "/partition-wikidata.out.ipynb",
    parameters=dict(
        wikidata_input_path = os.environ["OUT"] + "/all.tsv.gz",
        wikidata_parts_path = os.environ["OUT"] + "/parts",
        temp_folder_path = os.environ["OUT"] + "/parts/temp",
        sort_extras = "--buffer-size 30% --temporary-directory $OUT/parts/temp",
        verbose = False
    )
)
;

In [ ]:
pm.execute_notebook(
    os.environ["USECASE_DIR"] + "/Wikidata Useful Files.ipynb",
    os.environ["TEMP"] + "/Wikidata Useful Files Out.ipynb",
    parameters=dict(
        output_path = os.environ["OUT"],
        output_folder = "useful_files",
        temp_folder = "temp.useful_files",
        wiki_root_folder = os.environ["OUT"] + "/parts/",
        cache_path = os.environ["OUT"] + "/temp.useful_files",
        languages = 'en',
        compute_pagerank = True,
        delete_database = False
    )
)
;

## Sanity checks

- After removing classes, check that the class does not occur in the resulting claims file. 
- After protecting classes, check the class was occurs in the resulting claims file.

Check that the class we removed `Q34508` was removed from claims file

In [ ]:
!{kypher} -i {out}/claims.tsv.gz \
--match '(n1:Q34508)-[l]->(n2)' \
--limit 10 \
| col

In [ ]:
!{kypher} -i {out}/claims.tsv.gz \
--match '(n1:P131)-[l]->(n2)' \
--limit 10 \
| col

## Compute the derived files using the `Wikidata Useful Files` Jupyter notebook

Compute `claims.wikibase-item.tsv.gz` which would be computed by the Wikidata partitioner, but we are not using it here yet

In [ ]:
!zcat < "{datatypes}" | head | col

In [ ]:
!{kypher} -i {out}/claims.tsv.gz -i "{datatypes}" -o {out}/claims.wikibase-item.tsv.gz \
--match 'claims: (n1)-[l {label: p}]->(n2), datatypes: (p)-[:datatype]->(:`wikibase-item`)' \
--return 'l as id, n1 as node1, p as label, n2 as node2' \
--order-by 'l' 

To compute the derived files we use papermill to run the `Wikidata Useful Files` notebook.

In [ ]:
pm.execute_notebook(
    notebooks_folder + useful_files_notebook,
    temp + "/useful_files_notebook_output.ipynb",
    parameters=dict(
        output_path=output_path,
        output_folder=output_folder,
        temp_folder=temp_folder,
        wiki_root_folder=wiki_root_folder,
        claims_file="claims.tsv.gz",
        label_file="labels.en.tsv.gz",
        alias_file="aliases.en.tsv.gz",
        description_file="descriptions.en.tsv.gz",
        item_file="claims.wikibase-item.tsv.gz",
        cache_path=cache_path,
        delete_database=delete_database,
        compute_pagerank=compute_pagerank
    )
)

Look at the columns so we know how to construct the kypher query

## Summary of results

In [ ]:
!ls -lh {out}/*wikidataos.*

In [ ]:
!zcat < {out}/wikidataos.all.tsv.gz | wc

## Verification

The edges file must contain edges for properties, this is not the case on 2020-11-10`


In [ ]:
!{kgtk} -i "{claims}" \
--match '(:P10)-[l]->(n2)' \
--limit 10

## concatenate files to get the `all` file

In [ ]:
lad = []
if 'en' not in languages:
    languages.append('en')
for lang in languages:
    lad.append(f"{out}/labels.{lang}.tsv.gz")
    lad.append(f"{out}/aliases.{lang}.tsv.gz")
    lad.append(f"{out}/descriptions.{lang}.tsv.gz")
lad_file_list = " ".join(lad)

In [ ]:
!kgtk cat -i {out}/claims.tsv.gz \
{lad_file_list} \
{out}/qualifiers.tsv.gz \
{out}/metadata.pagerank.undirected.tsv.gz \
{out}/metadata.pagerank.directed.tsv.gz \
{out}/metadata.in_degree.tsv.gz \
{out}/metadata.out_degree.tsv.gz \
-o {out}/wikidataos.all.tsv.gz

## concatenate files to get the `all for triples` file


In [ ]:
!kgtk cat -i $OUT/wikidataos.all.tsv.gz \
$OUT/derived.P31.tsv.gz \
$OUT/derived.P279.tsv.gz \
$OUT/derived.isa.tsv.gz \
$OUT/derived.P279star.tsv.gz \
-o $OUT/wikidataos.all.for.triples.tsv.gz

## Filter out `novalue`, `somevalue` and `P9`

In [ ]:
!kgtk filter -i $OUT/wikidataos.all.for.triples.tsv.gz \
    -o $OUT/wikidataos.all.for.triples.filtered.tsv.gz \
    -p ';;somevalue,novalue,P9' --invert

## Add ids for any edge with missing id

In [ ]:
!kgtk add-id -i $OUT/wikidataos.all.for.triples.filtered.tsv.gz \
-o $OUT/wikidataos.all.for.triples.filtered.id.tsv.gz \
--id-style wikidata

## Sort by `id`

In [ ]:
!kgtk sort2 -i $OUT/wikidataos.all.for.triples.filtered.id.tsv.gz \
-o $OUT/wikidataos.all.for.triples.filtered.id.sorted.tsv.gz 
-c id